In [2]:
import os
os.chdir('E:/thuc/doan2/data')  # Đổi thư mục làm việc

In [3]:
# Import các thư viện cần thiết
import os
import numpy as np
import tensorflow as tf
from keras import layers, Model, Input
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt


In [4]:
# Cấu hình đường dẫn
pairs_dir = "pairs"
processed_dir = "processed"

# Load cặp ảnh và nhãn
pairs = np.load(os.path.join(pairs_dir, "pairs.npy"), allow_pickle=True)
labels = np.load(os.path.join(pairs_dir, "labels.npy"))

print(f"Số lượng cặp ảnh: {len(pairs)}")
print(f"Ví dụ cặp ảnh đầu tiên: {pairs[0]}, Nhãn: {labels[0]}")


Số lượng cặp ảnh: 2376
Ví dụ cặp ảnh đầu tiên: ['processed\\Akshay Kumar\\Akshay Kumar_0.jpg'
 'processed\\Akshay Kumar\\Akshay Kumar_1.jpg'], Nhãn: 1


In [5]:
# Hàm load ảnh và chuẩn hóa
def load_image(image_path, target_size=(160, 160)):
    image = Image.open(image_path).convert("RGB")
    image = image.resize(target_size)
    image = np.array(image) / 255.0  # Chuẩn hóa giá trị pixel về [0, 1]
    return image

# Chuẩn bị dữ liệu
X1 = []
X2 = []
y = []

for pair, label in zip(pairs, labels):
    img1, img2 = pair
    X1.append(load_image(img1))
    X2.append(load_image(img2))
    y.append(label)

X1 = np.array(X1)
X2 = np.array(X2)
y = np.array(y)

print(f"Dữ liệu X1: {X1.shape}, X2: {X2.shape}, y: {y.shape}")


Dữ liệu X1: (2376, 160, 160, 3), X2: (2376, 160, 160, 3), y: (2376,)


In [6]:
# Chia dữ liệu thành train và test
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

print(f"Số lượng mẫu train: {len(X1_train)}, Số lượng mẫu test: {len(X1_test)}")


Số lượng mẫu train: 1900, Số lượng mẫu test: 476


In [7]:
# Hàm tạo mô hình cơ bản để trích xuất đặc trưng
def create_base_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), activation='relu')(input_layer)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    return Model(input_layer, x)

# Tạo mô hình cơ bản
input_shape = (160, 160, 3)
base_model = create_base_model(input_shape)
base_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 158, 158, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 79, 79, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 77, 77, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 184832)            0     

In [8]:
# Xây dựng mô hình Siamese
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

encoded_a = base_model(input_a)
encoded_b = base_model(input_b)

# Khoảng cách L1 giữa hai vector đặc trưng
l1_distance = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([encoded_a, encoded_b])
output_layer = layers.Dense(1, activation='sigmoid')(l1_distance)

# Kết nối đầu vào và đầu ra
siamese_model = Model([input_a, input_b], output_layer)
siamese_model.summary()

# Biên dịch mô hình
siamese_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 128)          23734272    ['input_2[0][0]',                
                                                                  'input_3[0][0]']          

In [9]:
import os
os.chdir('E:/thuc/doan2')  # Đổi thư mục làm việc

In [10]:
# Huấn luyện mô hình
history = siamese_model.fit(
    [X1_train, X2_train], y_train,
    validation_data=([X1_test, X2_test], y_test),
    batch_size=32,
    epochs=10
)

# Lưu mô hình
MODEL_PATH = "models/siamese_model.h5"
os.makedirs("models", exist_ok=True)
siamese_model.save(MODEL_PATH)
print(f"Mô hình đã được lưu tại {MODEL_PATH}")


Epoch 1/10
60/60 [==============================] - 153s 2s/step - loss: 0.1495 - accuracy: 0.9879 - val_loss: 0.1229 - val_accuracy: 0.9895
Epoch 2/10
60/60 [==============================] - 145s 2s/step - loss: 0.0798 - accuracy: 0.9879 - val_loss: 0.1581 - val_accuracy: 0.9895
Epoch 3/10
60/60 [==============================] - 142s 2s/step - loss: 0.0862 - accuracy: 0.9879 - val_loss: 0.0901 - val_accuracy: 0.9895
Epoch 4/10
60/60 [==============================] - 142s 2s/step - loss: 0.0685 - accuracy: 0.9879 - val_loss: 0.0793 - val_accuracy: 0.9895
Epoch 5/10
60/60 [==============================] - 150s 2s/step - loss: 0.0637 - accuracy: 0.9879 - val_loss: 0.1018 - val_accuracy: 0.9895
Epoch 6/10
60/60 [==============================] - 146s 2s/step - loss: 0.0600 - accuracy: 0.9879 - val_loss: 0.0744 - val_accuracy: 0.9895
Epoch 7/10
60/60 [==============================] - 148s 2s/step - loss: 0.0547 - accuracy: 0.9879 - val_loss: 0.0694 - val_accuracy: 0.9895
Epoch 8/10
60

In [11]:
# Đánh giá mô hình trên tập test
loss, accuracy = siamese_model.evaluate([X1_test, X2_test], y_test)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")


15/15 [==============================] - 8s 556ms/step - loss: 0.0985 - accuracy: 0.9895
Loss: 0.0985, Accuracy: 0.9895
